In [1]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.200 🚀 Python-3.10.13 torch-2.0.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
Setup complete ✅ (16 CPUs, 7.4 GB RAM, 94.3/1006.9 GB disk)


In [2]:
from ultralytics import YOLO

In [3]:
# Transfer Learning: Fine Tunning
# Load the pre-trained YOLOv8 model
# model = YOLO(model="../models/yolov8/yolov8n.pt")
model = YOLO(model="yolov8n.pt")

100%|██████████| 6.23M/6.23M [00:05<00:00, 1.21MB/s]


### Model Info


In [4]:
model.info()

YOLOv8n summary: 225 layers, 3157200 parameters, 0 gradients, 8.9 GFLOPs


(225, 3157200, 0, 8.8575488)

In [5]:
model.model

DetectionModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): C2f(
      (cv1): Conv(
        (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
    

In [6]:
classes = []
components = []

count = 1
for index, info in enumerate(model.named_parameters()):
    # info[0]: name
    # info[1]: params
    if index == 0:
        print(f"name: {info[0]} \nparams: {info[1][0]}")

    if count == 1 and info[0].split(".")[2] == "conv":
        count = 0
        print(info[1].required_grade)

    classes.append(info[0].split(".")[2])
    components.append(info[0].split(".")[3])

set(classes), set(components)

name: model.model.0.conv.weight 
params: tensor([[[-0.1152, -0.0106,  0.1069],
         [-0.3682,  0.0334,  0.3875],
         [-0.3372, -0.0137,  0.3120]],

        [[-0.1423,  0.0098,  0.1522],
         [-0.4900,  0.0279,  0.4602],
         [-0.3726, -0.0028,  0.3481]],

        [[-0.0499, -0.0149,  0.0249],
         [-0.2137,  0.0329,  0.1938],
         [-0.1534,  0.0311,  0.1565]]])


({'0',
  '1',
  '12',
  '15',
  '16',
  '18',
  '19',
  '2',
  '21',
  '22',
  '3',
  '4',
  '5',
  '6',
  '7',
  '8',
  '9'},
 {'bn', 'conv', 'cv1', 'cv2', 'cv3', 'dfl', 'm'})

### Model Freeze


In [7]:
layer_names = []
for index, info in enumerate(model.named_parameters()):
    # info[0]: name
    # info[1]: params
    layer_names.append(info[0])

In [8]:
# Layers to be freeze
count_last_layers_freeze = 10
to_freeze_layer_names = layer_names[
    -count_last_layers_freeze - 1 : -1
]  # -1: don't freeze output layer `model.model.22.dfl.conv.weight`

to_freeze_layer_names

['model.model.22.cv3.1.2.weight',
 'model.model.22.cv3.1.2.bias',
 'model.model.22.cv3.2.0.conv.weight',
 'model.model.22.cv3.2.0.bn.weight',
 'model.model.22.cv3.2.0.bn.bias',
 'model.model.22.cv3.2.1.conv.weight',
 'model.model.22.cv3.2.1.bn.weight',
 'model.model.22.cv3.2.1.bn.bias',
 'model.model.22.cv3.2.2.weight',
 'model.model.22.cv3.2.2.bias']

In [9]:
# Freezing the layer
for index, info in enumerate(model.named_parameters()):
    # info[0]: name
    # info[1]: params

    info[1].requires_grad = True  # train all layers
    for name in to_freeze_layer_names:
        if info[0] == name:
            info[1].requires_grad = False  # Freeze the layers

In [10]:
def freeze_layers_fn(trainer):
    model = trainer.model
    layer_names = []

    for index, info in enumerate(model.named_parameters()):
        # info[0]: name
        # info[1]: params
        layer_names.append(info[0])

    # Layers to be freeze
    count_last_layers_freeze = 10
    to_freeze_layer_names = layer_names[
        -count_last_layers_freeze - 1 : -1
    ]  # -1: don't freeze output layer `model.model.22.dfl.conv.weight`

    for index, info in enumerate(model.named_parameters()):
        # info[0]: name
        # info[1]: params

        info[1].requires_grad = True  # train all layers
        for name in to_freeze_layer_names:
            if info[0] == name:
                info[1].requires_grad = False  # Freeze the layers

In [11]:
import torch

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [13]:
# Replace the final classification layer of the pre-trained YOLOv8 model with a new,
# randomly initialized classification layer that has the number of
# output classes equal to the number of classes in your new dataset

# model.nc = 2
# model.add_callback("on_train_start", freeze_layers_fn)  # callback for freeze layers

In [14]:
# Use the model
model.train(
    data="../datasets/config.yaml",
    task="detect",
    batch=16,
    device=device,
    imgsz=640,
    pretrained=True,
    rect=False,
    freeze=to_freeze_layer_names,
    val=True,
    # save_dir="results/detect/train", # not working
    seed=14,
    epochs=15,
)  # train the model

engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=../datasets/config.yaml, epochs=15, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=14, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=['model.model.22.cv3.1.2.weight', 'model.model.22.cv3.1.2.bias', 'model.model.22.cv3.2.0.conv.weight', 'model.model.22.cv3.2.0.bn.weight', 'model.model.22.cv3.2.0.bn.bias', 'model.model.22.cv3.2.1.conv.weight', 'model.model.22.cv3.2.1.bn.weight', 'model.model.22.cv3.2.1.bn.bias', 'model.model.22.cv3.2.2.weight', 'model.model.22.cv3.2.2.bias'], overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f94cca5b9a0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

In [15]:
# Predict with the model

test_imgs = "../datasets/annotated/test"
results_detection = model.predict(
    source=test_imgs,
    task="detect",
    model="runs/detect/train/weights/best.pt",
)


image 1/10 /home/adam-al-rahman/.university/upes/remx_yolov8/remx/../datasets/annotated/test/axis-deer-20231020014502-odaw.jpg: 640x640 1 axis-deer, 43.1ms
image 2/10 /home/adam-al-rahman/.university/upes/remx_yolov8/remx/../datasets/annotated/test/axis-deer-20231020014502-rlqi.jpg: 640x640 1 axis-deer, 50.5ms
image 3/10 /home/adam-al-rahman/.university/upes/remx_yolov8/remx/../datasets/annotated/test/axis-deer-20231020014502-splw.jpg: 640x640 1 axis-deer, 65.0ms
image 4/10 /home/adam-al-rahman/.university/upes/remx_yolov8/remx/../datasets/annotated/test/axis-deer-20231020014502-uosy.jpg: 640x640 1 axis-deer, 48.8ms
image 5/10 /home/adam-al-rahman/.university/upes/remx_yolov8/remx/../datasets/annotated/test/axis-deer-20231020014502-xjqz.jpg: 640x640 1 axis-deer, 46.3ms
image 6/10 /home/adam-al-rahman/.university/upes/remx_yolov8/remx/../datasets/annotated/test/elephant-20231020014502-mzsq.jpg: 640x640 3 elephants, 50.8ms
image 7/10 /home/adam-al-rahman/.university/upes/remx_yolov8/rem

In [29]:
# boxes coordinate
# results_detection[0].boxes.xyxy
no_test_img = 10
test_img_data_boxes = []
for i in range(no_test_img):
  test_img_data_boxes.append(results_detection[i].boxes)

In [36]:
for i in test_img_data_boxes:
  print(f"\nCLASS: {i.cls}\n-----")
  print(i)
  print("----------------------------------------------")


CLASS: tensor([0.], device='cuda:0')
-----
ultralytics.engine.results.Boxes object with attributes:

cls: tensor([0.], device='cuda:0')
conf: tensor([0.9941], device='cuda:0')
data: tensor([[253.6622,   0.0000, 530.1868, 337.3139,   0.9941,   0.0000]], device='cuda:0')
id: None
is_track: False
orig_shape: (640, 640)
shape: torch.Size([1, 6])
xywh: tensor([[391.9245, 168.6570, 276.5245, 337.3139]], device='cuda:0')
xywhn: tensor([[0.6124, 0.2635, 0.4321, 0.5271]], device='cuda:0')
xyxy: tensor([[253.6622,   0.0000, 530.1868, 337.3139]], device='cuda:0')
xyxyn: tensor([[0.3963, 0.0000, 0.8284, 0.5271]], device='cuda:0')
----------------------------------------------

CLASS: tensor([0.], device='cuda:0')
-----
ultralytics.engine.results.Boxes object with attributes:

cls: tensor([0.], device='cuda:0')
conf: tensor([0.9620], device='cuda:0')
data: tensor([[155.4162, 291.4303, 325.4197, 369.7431,   0.9620,   0.0000]], device='cuda:0')
id: None
is_track: False
orig_shape: (640, 640)
shape: 

In [18]:
path = model.export(format="onnx")

Ultralytics YOLOv8.0.200 🚀 Python-3.10.13 torch-2.0.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)

PyTorch: starting from 'runs/detect/train/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 6, 8400) (6.0 MB)

ONNX: starting export with onnx 1.14.1 opset 17...
ONNX: export success ✅ 0.7s, saved as 'runs/detect/train/weights/best.onnx' (11.7 MB)

Export complete (0.8s)
Results saved to /home/adam-al-rahman/.university/upes/remx_yolov8/remx/runs/detect/train/weights
Predict:         yolo predict task=detect model=runs/detect/train/weights/best.onnx imgsz=640  
Validate:        yolo val task=detect model=runs/detect/train/weights/best.onnx imgsz=640 data=../datasets/config.yaml  
Visualize:       https://netron.app


============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

